In [1]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
# don't need the below cell but it is good to see what the bassis of starting up a web driver is 

In [3]:
# url = 'https://untappd.com/brewery/top_rated?country=united-states&brewery_type=regional_brewery'
# driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
# driver.get(url)   #activates url (brings website up)

In [4]:
# all these urls are breweries' beer lists
urls = ['https://untappd.com/VanishBeer/beer','https://untappd.com/TrilliumBrewing/beer']#, 
#         'https://untappd.com/treehousebrewco/beer',
#         'https://untappd.com/SaltyTurtleBeerCompany/beer', 'https://untappd.com/BearChaseBrewingCompany/beer','https://untappd.com/TexasLeaguer/beer',
#         'https://untappd.com/WillowParkBrewing/beer','https://untappd.com/PipsMeadery/beer','https://untappd.com/EskerHartBrewingCompany/beer',
#         'https://untappd.com/prolyfk/beer','https://untappd.com/Marlobobo/beer', 'https://untappd.com/EmperorsBrewery/beer',
#         'https://untappd.com/SideProject/beer', 'https://untappd.com/troobado/beer', 'https://untappd.com/w/private-press-brewing/472291/beer', 
#         'https://untappd.com/w/fidens-brewing-co/426930/beer', 'https://untappd.com/MindfulAles/beer', 'https://untappd.com/rootandbranchbrewing/beer'
#         'https://untappd.com/WeaverHollowBrewery/beer', 'https://untappd.com/FreakFolkBier/beer', 'https://untappd.com/HorusAles/beer', 
#         'https://untappd.com/CaseyBrewingBlending/beer', 'https://untappd.com/HouseofFermentology/beer', 'https://untappd.com/treehousebrewco/beer', 
#         'https://untappd.com/barclaybrewingco/beer', 'https://untappd.com/Alchemist_Beer/beer', 'https://untappd.com/HillFarmsteadBrewery/beer',
#         'https://untappd.com/monkishbrewing/beer','https://untappd.com/topplinggoliathbrewing/beer', 'https://untappd.com/OldNationBrewingCompany/beer',
#         'https://untappd.com/PrairieArtisanAles/beer', 'https://untappd.com/fiddlehead/beer', 'https://untappd.com/NewEnglandBrewingCo/beer', 
#         'https://untappd.com/fremontbrewing/beer', 'https://untappd.com/RevisionBrewingCompany/beer', 'https://untappd.com/JackieOsBrewery/beer',
#         'https://untappd.com/westbrookbrewing/beer', 'https://untappd.com/creaturecomforts/beer', 'https://untappd.com/LaCumbreBrewingCompany/beer',
#         'https://untappd.com/SurlyBrewingCompany/beer','https://untappd.com/pfriembeer/beer','https://untappd.com/RevolutionBrewingChicago/beer',
#         'https://untappd.com/NewGlarusBrewingCompany/beer','https://untappd.com/halfacrebeer/beer','https://untappd.com/TheStarkeller/beer',
#         'https://untappd.com/HenHouseBrewing/beer','https://untappd.com/MelvinBrewing/beer','https://untappd.com/AlpineBeerCo/beer',
#         'https://untappd.com/Rhinegeist/beer','https://untappd.com/hardywood/beer', 'https://untappd.com/portcitybrewing/beer'] 

In [5]:
print('Number of different Breweries: ',len(urls))
#I accidenty added a meadery, that can either be good or bad tbh but just noting tgis so I don't forget it

Number of different Breweries:  2


In [6]:
name_ls = []
style_ls = []
desc_ls = []
abv_ls = []
ibu_ls = []

### Show details
scrolls through the page and clicks 'show more' to get the full description

In [7]:
def show_details(items):
    #this loop goes through all the beers in the list and clicks "show more" to get the full description
    for i in range(len(items)):
        beer_det = items[i].find_element_by_css_selector('div.beer-details')
        beer_det.location_once_scrolled_into_view
        try:
            beer_det.find_element_by_css_selector('a.read-more-beerlist.track-click').click()
        except NoSuchElementException:
            continue
        beer_det.location_once_scrolled_into_view

### Get Beer Details
This function goes through the list of beers and gets all the information from them and puts them in a list

In [67]:
def get_beer_details(items):
    for beer in items:   #goes through each beer in the "list" and gets the name, style, IBU, abv, and description (half)
        try:
            name = beer.find_element_by_css_selector('p.name').text    #extracts name of beer
            style = beer.find_element_by_css_selector('p.style').text   #extracts style of beer
            desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[4]').text[:-11]   #gets the full description and also cuts off the 'show more'
            if (desc=='')|(desc==' '): #if desc isn't long enough to have a "read More" then get the other desc which is stored where the half one is
                desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[3]').text[:-11]
                if (desc=='')|(desc==' '):    # in some edge cases the value desc is still going to be blank
                    print('test')
                    desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[5]').text[:-11]
            if 'no longer being produced' in desc:
                desc = beer.find_element_by_xpath('.//*[@class="beer-details"]/p[6]').text   # if the beer is no longer being produced
            abv = beer.find_element_by_css_selector('div.details-item.abv').text   #extracts abv of beer
            ibu = beer.find_element_by_css_selector('div.details-item.ibu').text   #extracts ibu of beer
            
        except NoSuchElementException:
            abv = beer.find_element_by_css_selector('p.abv').text   # part of the way through the process the namespace 
            ibu = beer.find_element_by_css_selector('p.ibu').text   # changes for ibu and abv so this handles that

        name_ls.append(name)     #these add all the attributes to a list that will eventually be set to a df
        style_ls.append(style)
        desc_ls.append(desc)    
        abv_ls.append(abv)             
        ibu_ls.append(ibu)

        print("Name: ", name,'\nStyle:',style,'\nABV: ',abv,'\nIBU: ',ibu,'\nDescription: ',desc,'\n\n------------\n')

In [23]:
for i in range(len(urls)):   #goes through all the urls/breweries
    url=urls[i]    #sets url to new brewery
    driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
    driver.get(url)   #activates url (brings website up)
    beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
    
    show_details(beers)  #method clicks show more on each beer
    get_beer_details(beers)

C:\Users\612456\AppData\Local\Temp\2\ipykernel_1192\1246772369.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
C:\Users\612456\AppData\Local\Temp\2\ipykernel_1192\1246772369.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers


Name:  Ghost Fleet 
Style: IPA - American 
ABV:  7.1% ABV 
IBU:  N/A IBU 
Description:  This is our house IPA. It is hugely hoppy with fruity aromatics from Amarillo and Simcoe hops. A hazy and unfiltered beer with a smooth finish. 

------------

Name:  Fat Boys 
Style: IPA - Imperial / Double 
ABV:  8.5% ABV 
IBU:  N/A IBU 
Description:  This is our Double IPA. It features bold fruity aromatics commonly associate with American IPAs. It contains a ton of hops including Citra and Equinox. As far as DIPAs go this one is light-bodied with an easy finish 

------------

Name:  Wraith 
Style: Wheat Beer - Witbier / Blanche 
ABV:  5.3% ABV 
IBU:  N/A IBU 
Description:  Wraith is a Belgian Style Witbier that provides you with notes of citrus, chamomile and coriander. 

------------

Name:  Lucketts Calling 
Style: IPA - American 
ABV:  6.3% ABV 
IBU:  N/A IBU 
Description:  A hazy IPA with big fruity hop flavors and almost no bitterness. 

------------

Name:  Beyond The Farm 
Style: IPA - A

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: unable to send message to renderer
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0029B8F3+2406643]
	Ordinal0 [0x0022AF31+1945393]
	Ordinal0 [0x0011C748+837448]
	Ordinal0 [0x0010C9B3+772531]
	Ordinal0 [0x0010CDC8+773576]
	Ordinal0 [0x00118BE0+822240]
	Ordinal0 [0x0010E0BD+778429]
	Ordinal0 [0x0010EC9A+781466]
	Ordinal0 [0x0010E2CA+778954]
	Ordinal0 [0x0010D8A6+776358]
	Ordinal0 [0x0010C7C5+772037]
	Ordinal0 [0x0010CC9D+773277]
	Ordinal0 [0x0011DF4A+843594]
	Ordinal0 [0x00174D3D+1199421]
	Ordinal0 [0x0016427C+1131132]
	Ordinal0 [0x00174682+1197698]
	Ordinal0 [0x00164096+1130646]
	Ordinal0 [0x0013E636+976438]
	Ordinal0 [0x0013F546+980294]
	GetHandleVerifier [0x00509612+2498066]
	GetHandleVerifier [0x004FC920+2445600]
	GetHandleVerifier [0x00334F2A+579370]
	GetHandleVerifier [0x00333D36+574774]
	Ordinal0 [0x00231C0B+1973259]
	Ordinal0 [0x00236688+1992328]
	Ordinal0 [0

#### the above cell takes ~40 mins to run 50 breweries

### Data is recorded in the lists now add to df

In [311]:
#create df
df = pd.DataFrame()
df['Name'] = name_ls
df['style'] = style_ls
df['IBU'] = ibu_ls
df['ABV'] = abv_ls
df['Description'] = desc_ls
df

,Name,style,IBU,ABV,Description
0,Ghost Fleet,IPA - American,N/A IBU,7.1% ABV,This is our house IPA. It is hugely hoppy with...
1,Fat Boys,IPA - Imperial / Double,N/A IBU,8.5% ABV,This is our Double IPA. It features bold fruit...
2,Wraith,Wheat Beer - Witbier / Blanche,N/A IBU,5.3% ABV,Wraith is a Belgian Style Witbier that provide...
3,Lucketts Calling,IPA - American,N/A IBU,6.3% ABV,A hazy IPA with big fruity hop flavors and alm...
4,Beyond The Farm,IPA - American,N/A IBU,6.8% ABV,"Relying on the citrusy Summit, Citra, and Cent..."
...,...,...,...,...,...
1209,Ideaal Tripel,Belgian Tripel,35 IBU,8.5% ABV,Drawing inspiration from Belgium’s monastic br...
1210,Hazy ipa,IPA - New England / Hazy,35 IBU,6.8% ABV,"Brewed to highlight hop aroma and flavor, Hazy..."
1211,Star Sailor®,IPA - White,40 IBU,6.2% ABV,"Exploring the limits of IPAs, Star Sailor® Whi..."
1212,German Pilsner,Pilsner - German,35 IBU,4.6% ABV,"Looking to Germany for inspiration, our German..."


In [312]:
#get rid of all beers that don't have all 3 major things I need
df = df[(df['IBU']!='N/A IBU')&(df['ABV']!='N/A ABV')&(df['Description']!='')]

In [325]:
df

,Name,style,IBU,ABV,Description
26,Vicinity,IPA - Imperial / Double,65,8,Originally brewed in celebration of the first ...
33,Galaxy Dry Hopped Fort Point,Pale Ale - American,45,6.6,This version of our signature American pale al...
47,Launch Beer,Pale Ale - American,68,5.7,First brewed to commemorate the grand opening ...
50,Julius,IPA - American,72,6.8,"Bursting with pungent American hops, Julius - ..."
51,Haze,IPA - Imperial / Double,90,8.2,Our Double IPA! We smell a ton of peach on the...
...,...,...,...,...,...
1208,Mexican Dark Lager,Lager - Vienna,24,5,Port City’s Mexican Dark Lager draws inspirati...
1209,Ideaal Tripel,Belgian Tripel,35,8.5,Drawing inspiration from Belgium’s monastic br...
1210,Hazy ipa,IPA - New England / Hazy,35,6.8,"Brewed to highlight hop aroma and flavor, Hazy..."
1211,Star Sailor®,IPA - White,40,6.2,"Exploring the limits of IPAs, Star Sailor® Whi..."


In [324]:
df['IBU'] = df['IBU'].str[:-4]

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\849542697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].str[:-4]


In [321]:
df['ABV'] = df['ABV'].str[:-5]

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\2652638095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].str[:-5]


In [326]:
df['ABV'] = df['ABV'].astype(float)

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\2667934893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ABV'] = df['ABV'].astype(float)


In [327]:
df['IBU'] = df['IBU'].astype(int)

C:\Users\612456\AppData\Local\Temp\7\ipykernel_18756\3279095354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IBU'] = df['IBU'].astype(int)


In [328]:
df

,Name,style,IBU,ABV,Description
26,Vicinity,IPA - Imperial / Double,65,8.0,Originally brewed in celebration of the first ...
33,Galaxy Dry Hopped Fort Point,Pale Ale - American,45,6.6,This version of our signature American pale al...
47,Launch Beer,Pale Ale - American,68,5.7,First brewed to commemorate the grand opening ...
50,Julius,IPA - American,72,6.8,"Bursting with pungent American hops, Julius - ..."
51,Haze,IPA - Imperial / Double,90,8.2,Our Double IPA! We smell a ton of peach on the...
...,...,...,...,...,...
1208,Mexican Dark Lager,Lager - Vienna,24,5.0,Port City’s Mexican Dark Lager draws inspirati...
1209,Ideaal Tripel,Belgian Tripel,35,8.5,Drawing inspiration from Belgium’s monastic br...
1210,Hazy ipa,IPA - New England / Hazy,35,6.8,"Brewed to highlight hop aroma and flavor, Hazy..."
1211,Star Sailor®,IPA - White,40,6.2,"Exploring the limits of IPAs, Star Sailor® Whi..."


In [240]:
# look at how I can get more beers for 
###Wheat Beer 207 more
###Porter 207 more
###Belgian 203 more
###Lager 191 more
###Pale Ale 178 more
###Wild/Sour Ale 155 more
###Stout 126 more

### new method to get more beers that aren't IPAs

In [80]:
url = 'https://untappd.com/beer/top_rated'
driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver
driver.get(url)   #activates url (brings website up)

C:\Users\612456\AppData\Local\Temp\1\ipykernel_14464\3467423471.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\612456\Downloads\chromedriver_win32\chromedriver.exe')     #sets up chrome driver


In [81]:
name_ls = []
style_ls = []
desc_ls = []
abv_ls = []
ibu_ls = []

In [82]:
## This is the right idea the names of the drivers are just different at the moment and aren't working
# this will be an easy fix, either change the names of the driver and ther variables below or
# copy the code above and bring it down here and just change the names in that 


# #this goes through each of the styles, clicks on them and skips IPAs
for i in range(2,226):
    item = 'option['+str(i)+']'
    menu = driver.find_element_by_css_selector('select#filter_picker')
    menu.click()  #clicks on the drop down menu
    style = menu.find_element_by_xpath('//*[@id="filter_picker"]/'+item)
    if 'IPA' not in style.text:
        style.click()
        beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers
        show_details(beers)
        get_beer_details(beers)
    driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  #scrolls back to the top of the page to click the next style
    menu.location_once_scrolled_into_view  #scrolls til the field is in view

C:\Users\612456\AppData\Local\Temp\1\ipykernel_14464\3201200849.py:9: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  menu = driver.find_element_by_css_selector('select#filter_picker')
C:\Users\612456\AppData\Local\Temp\1\ipykernel_14464\3201200849.py:14: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  beers = driver.find_elements_by_css_selector('div.beer-item')     #finds each individual beer and sets it to a "list" of beers


Name:  Längst (Blend #1) 
Style: Side Project Brewing 
ABV:  10% ABV 
IBU:  N/A IBU 
Description:  Längst is our Bourbon barrel-aged Adambier - a historic, strong, malty German ale that was brewed with 7 different German malts and fermented with a fruity yeast strain. Längst was aged for 15 months in a variety of bourbon barrels. Read Less 

------------

test
Name:  Adam from the Wood 
Style: Hair of the Dog Brewing Company 
ABV:  12% ABV 
IBU:  N/A IBU 
Description:  Adam From The Wood is our Adam aged in American Oak barrels. First released in 2000, and released again in November 2011 in 12oz bottles. This 12% beer has lots of the typical HOTD aromas: Caramel, brown sugar, tons of raisin and tobacco. Fig, date, and plum fruitiness in that order. This has a fairly strong earthy vinousness as well as oak vanilla 

------------

Name:  Cherry Adam from the Wood 
Style: Hair of the Dog Brewing Company 
ABV:  13.5% ABV 
IBU:  N/A IBU 
Description:  Cherry Adam from the Wood is aged with 

Name:  The Long, the Short & the Tall of It 
Style: Marble Beers Ltd 
ABV:  9.5% ABV 
IBU:  N/A IBU 
Description:  Brewed in collaboration with our friends at Brouwerij Kees and Brouwerij De Molen in the Netherlands, this beer is part of a set of three collaborations, all brewed with Lapsang Souchong tea. We have revived the ancient German Adambier style, which balances rich dark malts with a touch of smoke. Read Less 

------------

test
Name:  Wild Sour Series: Adambier 
Style: DESTIHL Brewery 
ABV:  9% ABV 
IBU:  40 IBU 
Description:  Adambier is a very complex, dark and historically obscure German-style sour ale. The initial impression of rich, dark chocolate malt with touches of fig-like and raisin-like flavors gives way to the caramel-toasty and lightly smoky malt character with low hop presence and bitterness, all sharply cut by a wild lactic sour finish and topped off with tannins from wood aging 

------------

test
Name:  Universal Mind 
Style: Beavertown 
ABV:  10.5% ABV 
IB

C:\Users\612456\AppData\Local\Temp\1\ipykernel_14464\3201200849.py:17: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  #scrolls back to the top of the page to click the next style


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0062B8F3+2406643]
	Ordinal0 [0x005BAF31+1945393]
	Ordinal0 [0x004AC748+837448]
	Ordinal0 [0x004AF154+848212]
	Ordinal0 [0x004AF012+847890]
	Ordinal0 [0x004AF98A+850314]
	Ordinal0 [0x005050C9+1200329]
	Ordinal0 [0x004F427C+1131132]
	Ordinal0 [0x00504682+1197698]
	Ordinal0 [0x004F4096+1130646]
	Ordinal0 [0x004CE636+976438]
	Ordinal0 [0x004CF546+980294]
	GetHandleVerifier [0x00899612+2498066]
	GetHandleVerifier [0x0088C920+2445600]
	GetHandleVerifier [0x006C4F2A+579370]
	GetHandleVerifier [0x006C3D36+574774]
	Ordinal0 [0x005C1C0B+1973259]
	Ordinal0 [0x005C6688+1992328]
	Ordinal0 [0x005C6775+1992565]
	Ordinal0 [0x005CF8D1+2029777]
	BaseThreadInitThunk [0x74FFFA29+25]
	RtlGetAppContainerNamedObjectPath [0x770E7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x770E7A4E+238]


The above cell is working, I just need to implement a wait to mae sure it is visible
or I need to scroll more not sure?

In [45]:
x = driver.find_element_by_xpath('//*[@id="slide"]/div[2]/div/div[1]/div/div[4]/div[1]/div[1]/p[4]')
print(x.text)

This beer is no longer being produced by the brewery


C:\Users\612456\AppData\Local\Temp\1\ipykernel_14464\846203888.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  x = driver.find_element_by_xpath('//*[@id="slide"]/div[2]/div/div[1]/div/div[4]/div[1]/div[1]/p[4]')


In [47]:
'no longer being produced' in x.text

True

In [16]:
# save the df to a csv that I can use later and don't have to run this every time
df.to_csv('untappd_beers.csv')